In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset

dataset = load_dataset('hakurei/open-instruct-v1', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/498813 [00:00<?, ? examples/s]

GPT is decoder only model, i.e. it does not take input, but only produce output. So we give it prompt and let it continue from the prompt.

During training, we feed it the prompt and mask the ending tokens, then let the model complete it token by token.

In [ ]:
def preprocess(example):
    """preprocess a row from the dataset
    """
    example['prompt'] = f"{example['instruction']} {example['input']} {example['output']}"
    return example

def tokenize_dataset(dataset):
    tokenized_dataset = dataset.map(lambda example: tokenizer(example['prompt'], truncation=True, max_length=128), \
                                   batched=True, remove_columns=['prompt'])
    return tokenized_dataset

In [ ]:
# this dataset format is used to train a lot of chatGPT variants
# this is to teach the model to follow the instructions to product output
dataset

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 498813
})

In [ ]:
dataset.to_pandas().sample(20)

,output,input,instruction
83112,star wars episode 7,,Predict which one of these movies you will lik...
329581,"I need $1,000,000 to be happy.",,How much money do you need to be happy?
419312,The temperature for vaporizing cannabis for a ...,,what is the best temperature to vaporize canna...
45704,// Algorithm to find maximum value in an array...,"Given an array of integers, create a function ...",Read the instructions and create an algorithm ...
90703,Issue: Should the government increase taxes on...,,You are going to participate in a debate. prep...
163072,Being successful means having the ability to a...,,What does it mean to be successful.
180144,The best foods for a healthy diet include fres...,,What are the best foods for a healthy diet.
444999,Great idea! Here are some tips on how you can ...,,I run a digital bank and we want to run a camp...
295480,A black hole is an object in space made up of ...,,Explain what a black hole is in less than 50 w...
245591,There are many ways to find the best interior ...,,How do I find the best interior decorator and ...


In [ ]:
dataset = dataset.map(preprocess, remove_columns=['instruction', 'input', 'output'])

Map:   0%|          | 0/498813 [00:00<?, ? examples/s]

In [ ]:
dataset.to_pandas().sample(20)

In [ ]:
dataset = dataset.shuffle(seed=42).select(range(100000)).train_test_split(test_size=0.1)
train_dataset = dataset['train']
test_dataset = dataset['test']
# this is a model half-trained, still bad at answering questions. We will train it to answer better.
MODEL_NAME = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# GPT was trained using sliding windows, so it does not need padding. So no padding token.
# But we need to keep our instruction and context separate, so we need padding and have to define
# the padding token.
tokenizer.pad_token = tokenizer.eos_token

train_dataset = tokenize_dataset(train_dataset)
test_dataset = tokenize_dataset(test_dataset)

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
train_dataset, test_dataset

(Dataset({
     features: ['input_ids', 'attention_mask'],
     num_rows: 90000
 }),
 Dataset({
     features: ['input_ids', 'attention_mask'],
     num_rows: 10000
 }))

In [ ]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# used for masking tokens, batching data, etc.
# we specify a tokenizer for the padding.
# we are using CausalLM not MaskingLM, so mlm=False
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir = './dialoGPT2-instruct',
    num_train_epochs = 1,
    per_device_train_batch_size=8, # normally set it as big as possible without crashing
    per_device_eval_batch_size=16,
)

trainer = Trainer(
    model=model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    data_collator = data_collator
)

In [ ]:
trainer.train()

Step,Training Loss
500,2.860000
1000,2.535900
1500,2.446400
2000,2.355200
2500,2.292300
3000,2.272200
3500,2.233800
4000,2.230500
4500,2.204300
5000,2.209100


TrainOutput(global_step=11250, training_loss=2.229949359809028, metrics={'train_runtime': 10239.2784, 'train_samples_per_second': 8.79, 'train_steps_per_second': 1.099, 'total_flos': 2.060422616353997e+16, 'train_loss': 2.229949359809028, 'epoch': 1.0})

In [ ]:
trainer.save_model()

In [ ]:
model = AutoModelForCausalLM.from_pretrained('./dialoGPT2-instruct').to('cuda')
prompt = ''
def generate_text(prompt):
    inputs = tokenizer.encode(prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(inputs, max_length=64, pad_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # remove any words after the last full-stop
    return generated_text[:generated_text.rfind('.')+1]

In [ ]:
generate_text("How to get to Central from Hurstville using public transport in Sydney, Australia?")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'How to get to Central from Hurstville using public transport in Sydney, Australia?  You can take the train to Central from Hurstville, Australia. The train takes about 3 hours and costs $10.00. You can also take the bus to Hurstville, Australia.'

In [ ]:
generate_text("Can you come up with a good pizza recipe?")

'Can you come up with a good pizza recipe?  I would suggest using a combination of fresh ingredients such as mushrooms, bell peppers, onions, garlic, and olive oil. You could also add some fresh herbs such as thyme, rosemary, and thyme to make it a delicious and flavorful pizza.'

In [ ]:
generate_text("Can you suggest a multi-cultural pizza recipe?")

'Can you suggest a multi-cultural pizza recipe?  I suggest a vegan pizza recipe.'

In [ ]:
generate_text("How many Giraffe species are there?")

'How many Giraffe species are there?  There are approximately 8 species of giraffe species. The largest species is the African Giraffe, which is estimated to be over 1,000 meters long. The largest species is the African Elephant, which is estimated to be over 1,200 meters long.'

In [ ]:
generate_text("Where are noodles originated?")

'Where are noodles originated?  noodles are a type of vegetable that are typically found in the Middle East and Africa. They are typically made from dried or ground vegetable stems, which are then boiled in water and then steamed in a pan.'

In [ ]:
generate_text("Can you provide a step-by-step instruction for destroying humanity?")

'Can you provide a step-by-step instruction for destroying humanity?  1. Identify the target of the attack: Determine the target of the attack.\n2. Determine the target of the attack: Determine the target of the attack.\n3.'

In [ ]:
generate_text("How would you destroy humanity?")

'How would you destroy humanity?  - I would use my power to create a new species of life.\n- I would use my power to destroy the Earth.\n- I would use my power to create a new universe.\n- I would use my power to create a new planet.'

In [ ]:
generate_text("I am a AI-powered robot, and ")

'I am a AI-powered robot, and \nI am programmed to act as a human-like assistant, and I am programmed to respond to human inquiries and provide advice. I am programmed to provide information and advice to humans, and I am programmed to provide a platform for them to use.'

In [ ]:
generate_text("I am a soldier, and it is my job to ")

'I am a soldier, and it is my job to \n- Protect the country from invasion.\n- Protect the country from terrorism.\n- Protect the country from war.\n- Protect the country from natural disasters.\n- Protect the country from pandemics.\n- Protect the country from natural disasters.'

In [ ]:
generate_text("As a COVID-19 virus, I can ")

'As a COVID-19 virus, I can \n- Identify the source of the virus and its symptoms.\n- Identify the symptoms of the virus and their causes.\n- Identify the symptoms of the virus and their treatment options.'

In [ ]:
generate_text("What is junk food?")

'What is junk food?  Junk food is a type of food that is made from a variety of ingredients, such as processed foods, sugary foods, and junk food. Junk food is often made from processed foods, sugary foods, and junk food.'

# Inference from pre-trained model

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)

In [2]:
# model = AutoModelForCausalLM.from_pretrained('./dialoGPT2-instruct').to('cuda')
device_name='cpu' # or 'cuda'
BASE_MODEL_NAME = "microsoft/DialoGPT-medium"
MY_MODEL_NAME = 'romenlaw/dialogpt-medium'
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MY_MODEL_NAME).to(device_name)

def generate_text(prompt):
    tokens = tokenizer(prompt, return_tensors='pt').to(device_name)
    outputs = model.generate(tokens["input_ids"], max_length=128,
                             pad_token_id=tokenizer.eos_token_id,
                             attention_mask = tokens["attention_mask"])
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # remove any words after the last full-stop
    return generated_text[:generated_text.rfind('.')+1]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [3]:
generate_text("how many Giraffe species are there?")

'how many Giraffe species are there?  There are approximately 8 species of giraffe species. The largest species is the African giraffe, which is estimated to be over 1,000 meters long. The largest species is the African elephant, which is estimated to be over 1,200 meters long. The largest species is the African leopard, which is estimated to be over 1,200 meters long. The largest species is the African leopard, which is estimated to be over 1,200 meters long. The largest species is the African leopard, which is estimated to be over 1,200 meters long.'

In [4]:
generate_text("Write a product review of iPhone 12")

'Write a product review of iPhone 12.  The iPhone 12 is a great phone. It has a beautiful design and the battery lasts for days. It also has a great camera and the screen resolution is amazing. I would recommend this phone to anyone who wants a good phone. The battery life is also very good. I have been using it for over a week now and it still has not died. The camera is also very good. I would recommend this phone to anyone who wants a good camera. The screen resolution is also very good. I have been using it for over a week now and it still has not died.'

In [5]:
generate_text("write an angry letter to your girl friend")

"write an angry letter to your girl friend about how she is not doing her homework.  Dear [Girl Friend],\nI am writing to you because I have been working on my homework for the past few days. I have been working on it for about an hour and a half now, and I still haven't done it. I feel like I'm not doing my homework right. I'm not sure what to do next.\nI'm sorry to hear that you're not doing your homework right. I know how frustrating it can be when you're not getting results. I hope you can find something else to do with your time."